In [1]:
import os

import torch
import torch.optim as optim
import torch.nn.functional as F

import Othello

from tqdm.auto import trange

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# Policy Gradient

$$
\max J(\pi)=\int_\tau p(\tau \mid \pi)R(\tau) =\mathbb E_{\tau\sim\pi}[R(\tau)]
$$

$$
\begin{align}\nabla_\theta J(\pi_\theta) &= \int_\tau \nabla_\theta p(\tau \mid \pi_\theta)R(\tau) \\%&= \int_\tau  p(\tau \mid \pi_\theta)\frac {\nabla_\theta p(\tau \mid \pi_\theta)}{ p(\tau \mid \pi_\theta)}R(\tau) \\ 
&= \mathbb E_{\tau\sim\pi_\theta}\left[\nabla_\theta \ln p(\tau \mid \pi_\theta)R(\tau)\right] \\ 
&= \mathbb E_{\tau\sim\pi_\theta}\left[\nabla_\theta \left(\ln \rho_0(S_0) + \sum_{t=0}^T\ln p(S_{t+1} \mid S_t,A_t) + \sum_{t=0}^T\ln \pi_\theta(A_t \mid S_t)\right)R(\tau)\right] \\ 
\nabla_\theta J(\pi_\theta) &= \mathbb E_{\tau\sim\pi_\theta}\left[ \sum_{t=0}^T \nabla_\theta \ln \pi_\theta(A_t \mid S_t)R(\tau)\right] \\\end{align}
$$
					

In [2]:

if torch.cuda.is_available():
    device = 'cuda'
else: device = 'cpu'

# the setting of game
n = 8
aim = "win" # "loss"

# model setting 
bone = "ResNet" # ResNet for lightweight model, ViT for large model
config = {"dims": [1, 64, 128, 256, 128, 64, 1], "kernel_size": 5}

# train parameters
n_epoch = 100000
batch_size = 32
lr = 1e-4

# Gradient policy Coefficients
beta = 0.5 # the weight of KL

path = f"PG_n={n}_beta={beta}_bone={bone}_{aim}"

In [3]:
envs = Othello.Envs(n, batch_size, device)

actor = Othello.Actor(n, bone, **config).to(device)

opt = optim.Adam(actor.parameters(), lr=lr) 

In [5]:
baseline = Othello.Actor(n, 'ResNet').to(device)
baseline.load_state_dict(
        torch.load(f"model/n={n}_beta=0.5_bone=ResNet_{aim}/epoch_bestVsRandom.pth")
    )
baseline.eval();

In [6]:
TIMESTAMP = f"{datetime.now():%Y-%m-%dT%H-%M-%S}"

writer = SummaryWriter(log_dir=f"tf_dir/{path}/{TIMESTAMP}/")

In [7]:
def PGloss(agents, envs, n, beta=0.1, aim="win"):
    
    if isinstance(agents, list):
        assert len(agents) == 2, "Othello requires two player" 
    else:
        agents = [agents, agents] # self-playing game
    
    for agent in agents:
        if isinstance(agent, torch.nn.Module):
            device = next(agent.parameters()).device
        
    batch_size = envs.batch_size
    rounds = n * (n + 1) 
    
    obs, reward, terminated, _, _ = envs.reset()

    actions = torch.zeros(batch_size, 2, rounds//2, device=device).type(torch.int64) # the chosen actions
    rewards = torch.zeros(batch_size, 2, rounds//2, device=device) # rewards
    terminateds = torch.zeros(batch_size, 2, rounds//2, device=device) # whether the game is terminated
    probs = torch.zeros(batch_size, 2, rounds//2, device=device) # the probs of the chosen actions
    KLs = torch.zeros(batch_size, 2, rounds//2, device=device) # KL control the distribution of probs on actions

    for i in range(rounds//2): 
        for p in range(2): 

            valid = obs[-1]

            agent = agents[p]

            prob = agent(obs).float() * valid # change the prob of invalid moves to 0

            # randomly choose the next move on the prob distributation
            actions[:, p, i] = torch.multinomial(prob, num_samples=1).view(-1)
            probs[:, p, i] = (F.one_hot(actions[:, p, i], num_classes=n*n+1) * prob).sum(-1)

            # save reward & terminated
            rewards[:, p, i] = reward
            terminateds[:, p, i] = terminated

            # cross entropy
            PriDis = ((1 - (valid == 0).sum(1) * 1e-6) / valid.sum(1))[:,None] * valid + 1e-6 * (valid == 0) # invalid move for 1e-6
            KLs[:, p, i] = (prob * (torch.log(prob + 1e-6) - torch.log(PriDis))).sum(1) # add 1e-6 to prevent -inf

            # next step
            act = F.one_hot(actions[:, p, i], num_classes=n*n+1)
            obs, reward, terminated, _, _ = envs.step(act, restart=False)

    # caculate hat_R
    R = torch.zeros(batch_size, 2, rounds//2, device=device)
    tmp = torch.zeros(batch_size, 2, device=device)

    for i in range(rounds//2-1, -1, -1): 
        for p in range(2): 

            R[:, p, i] = rewards[:, p, i] + (1 - terminateds[:, p, i]) * tmp[:, p] # Truncate the reward when the game is terminated
        
        tmp = R[:, :, i]

    # optimize
    if aim == "win":
        PGs = -R.detach() * torch.log(probs)
    elif aim == "loss":
        PGs = R.detach() * torch.log(probs)
    else: raise ValueError("aim must be 'win' or 'loss'")
    
    Loss = PGs.mean() + beta * KLs.mean()

    return Loss, {"PGloss": PGs[:, 0, :].mean().item(), "KLloss": KLs.mean().item()} # return Loss, and detailed loss of the black

In [8]:
@torch.no_grad()
def Test(agents, envs, n, times=10, strategy='biggest'):
    
    if isinstance(agents, list):
        assert len(agents) == 2, "Othello requires two players" 
    else:
        agents = [agents, agents] # self-playing game
    
    for agent in agents:
        if isinstance(agent, torch.nn.Module):
            device = next(agent.parameters()).device
    
    batch_size = envs.batch_size
    rounds = n * (n + 1) 
    
    blackWin = 0
    whiteWin = 0
    
    for k in range(times):
    
        obs, reward, _, _, _ = envs.reset()

        actions = torch.zeros(batch_size, 2, rounds//2, device=device).type(torch.int64) # the chosen actions
        rewards = torch.zeros(batch_size, 2, rounds//2, device=device)

        for i in range(rounds//2): 
            for p in range(2): 

                valid = obs[-1]

                agent = agents[p]

                prob = agent(obs).float() * valid # change the prob of invalid moves to 0

                if strategy == "biggest":
                    # choose the next move with the biggest prob
                    actions[:, p, i] = prob.max(-1)[1].view(-1)
                elif strategy == "random":
                    # randomly choose the next move on the prob distributation
                    actions[:, p, i] = torch.multinomial(prob, num_samples=1).view(-1)

                # save reward
                rewards[:, p, i] = reward

                # next step
                act = F.one_hot(actions[:, p, i], num_classes=n*n+1)
                obs, reward, terminated, _, _ = envs.step(act, restart=False)

        black_win = (rewards[:, 0].sum(-1) > 0).sum() / batch_size
        white_win = (rewards[:, 1].sum(-1) > 0).sum() / batch_size
        
        blackWin += black_win / times
        whiteWin += white_win / times
    
    tie = 1 - blackWin - whiteWin

    return {"Prob of black win": blackWin.item(), "Prob of white win": whiteWin.item(), "Prob of tie": tie.item()}

In [9]:
res_train = {'PGloss': 0, 'KLloss': 0}

for i in trange(n_epoch): 
    
    actor = actor.train()
    
    loss, detail = PGloss(actor, envs, n, beta=beta, aim=aim)
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    res_train['PGloss'] += detail['PGloss'] / 100
    res_train['KLloss'] += detail['KLloss'] / 100
        
    if i % (n_epoch // 1000) == (n_epoch // 1000) - 1 or i == 0:
        
        actor = actor.eval()
        
        res = Test([actor, envs.randomAction], envs, n, times=10)
        print('Black Agent, White Random:') 
        for key, value in res.items():
            print(f"{key}: {value:7.2%}", end='; ')
        print('\n')
        
        writer.add_scalar("Agent Black Win", res["Prob of black win"], i)
        
        res = Test([envs.randomAction, actor], envs, n, times=10)
        print('Black Random, White Agent:') 
        for key, value in res.items():
            print(f"{key}: {value:7.2%}", end='; ')
        print('\n')
        
        writer.add_scalar("Agent White Win", res["Prob of white win"], i)
        
        res = Test([actor, baseline], envs, n, times=10, strategy='random')
        print('Black Agent, White Baseline:') 
        for key, value in res.items():
            print(f"{key}: {value:7.2%}", end='; ')
        print('\n')
        
        writer.add_scalar("Agent Black Win (Baseline)", res["Prob of black win"], i)
        
        res = Test([baseline, actor], envs, n, times=10, strategy='random')
        print('Black Baseline, White Agent:') 
        for key, value in res.items():
            print(f"{key}: {value:7.2%}", end='; ')
        print('\n')
        
        writer.add_scalar("Agent White Win (Baseline)", res["Prob of white win"], i)
        
        for key, value in res_train.items():
            writer.add_scalar(key, value, i)
            res_train[key] = 0
            
    if i % (n_epoch // 40) == (n_epoch // 40) - 1:
            
        if not os.path.exists(f"model/{path}/{TIMESTAMP}"):
            os.makedirs(f"model/{path}/{TIMESTAMP}")
        
        torch.save(actor.state_dict(), f"model/{path}/{TIMESTAMP}/epoch{i}.pth")

  0%|          | 0/100000 [00:00<?, ?it/s]

Black Agent, White Random:
Prob of black win:  51.87%; Prob of white win:  45.94%; Prob of tie:   2.19%; 

Black Random, White Agent:
Prob of black win:  37.19%; Prob of white win:  58.75%; Prob of tie:   4.06%; 

Black Agent, White Baseline:
Prob of black win:  28.12%; Prob of white win:  69.06%; Prob of tie:   2.81%; 

Black Baseline, White Agent:
Prob of black win:  65.31%; Prob of white win:  30.94%; Prob of tie:   3.75%; 

Black Agent, White Random:
Prob of black win:  83.44%; Prob of white win:  14.37%; Prob of tie:   2.19%; 

Black Random, White Agent:
Prob of black win:  11.56%; Prob of white win:  86.25%; Prob of tie:   2.19%; 

Black Agent, White Baseline:
Prob of black win:  34.06%; Prob of white win:  62.19%; Prob of tie:   3.75%; 

Black Baseline, White Agent:
Prob of black win:  56.87%; Prob of white win:  39.06%; Prob of tie:   4.06%; 

Black Agent, White Random:
Prob of black win:  85.62%; Prob of white win:  13.75%; Prob of tie:   0.63%; 

Black Random, White Agent:
Pr